# Header

In [1]:
# 2021-03, Bruno Grossniklaus, https://github.com/it-gro
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

import pandas as pd
import pymongo

# Configuration

In [2]:
pd.set_option('precision', 2)
pd.set_option('max_rows', 30)
pd.set_option('max_colwidth', 100)
# pd.describe_option('max_rows')
# pd.describe_option('precision')
# pd.describe_option('max_colwidth')

HOST_mongo = 'localhost'
OPTIONS_mongo = ''
# OPTIONS_mongo = '?retryWrites=true&w=majority'
USER_mongo = ""
PASS_mongo = ""
if USER_mongo:
    credentials=f"{USER_mongo}:{PASS_mongo}@"
else:
    credentials=""

DB_CONNECT=f"mongodb://{credentials}{HOST_mongo}{OPTIONS_mongo}"
        
DB_ICNDB="imp_demo_icndb"
URL_API="http://api.icndb.com/jokes/random"
NUMEBR_OF_JOKES=100
COLL_SRC="src_jokes"
COLL_JOKES="jokes"

# Import

## V1

In [3]:
%%bash 

curl -sL http://api.icndb.com/jokes/random/100 | jq '.value' |
  mongoimport --db imp_demo_icndb --jsonArray --drop --collection src_jokes

cat <<EOF | mongo --quiet mongodb://localhost/imp_demo_icndb
  db["jokes"].drop()
  db["src_jokes"].renameCollection("jokes")
EOF

true
{ "ok" : 1 }


2021-03-20T17:48:10.522+0100	connected to: mongodb://localhost/
2021-03-20T17:48:10.523+0100	dropping: imp_demo_icndb.src_jokes
2021-03-20T17:48:10.816+0100	100 document(s) imported successfully. 0 document(s) failed to import.


##  V2

In [4]:
%%bash -s "{URL_API}" "{NUMEBR_OF_JOKES}" "{DB_CONNECT}" "{DB_ICNDB}" "{COLL_SRC}" "{COLL_JOKES}" 
request=$1/$2
connect=$3
db=$4
src_collection=$5
collection=$6

curl -sL ${request} | jq '.value' |
  mongoimport --db ${db} --jsonArray --drop --collection ${src_collection}

cat <<EOF | mongo --quiet ${connect}/${db}
  db["${collection}"].drop()
  db["${src_collection}"].renameCollection("${collection}")
EOF

true
{ "ok" : 1 }


2021-03-20T17:48:11.175+0100	connected to: mongodb://localhost/
2021-03-20T17:48:11.175+0100	dropping: imp_demo_icndb.src_jokes
2021-03-20T17:48:11.337+0100	100 document(s) imported successfully. 0 document(s) failed to import.


## V3

In [5]:
%%bash -s "{URL_API}" "{NUMEBR_OF_JOKES}" "{DB_CONNECT}" "{DB_ICNDB}" "{COLL_SRC}"
request=$1/$2
connect=$3
db=$4
src_collection=$5

curl -sL ${request} | jq '.value' |
  mongoimport --db ${db} --jsonArray --drop --collection ${src_collection}

cat <<'EOF' | mongo --quiet ${connect}/${db}
  db["src_jokes"].aggregate([
    {'$match': {"categories": {'$nin' : ["explicit"]}}},
    {'$project': {"_id": '$id', "joke": 1, "categories": 1}},
    {'$out': "jokes"}
  ])
  // db["src_jokes"].drop()
EOF


2021-03-20T17:48:11.494+0100	connected to: mongodb://localhost/
2021-03-20T17:48:11.496+0100	dropping: imp_demo_icndb.src_jokes
2021-03-20T17:48:11.645+0100	100 document(s) imported successfully. 0 document(s) failed to import.


# Connect

In [6]:
client = pymongo.MongoClient(f"mongodb://{credentials}{HOST_mongo}{OPTIONS_mongo}")
icndb = client[DB_ICNDB]

# Show Documents

In [7]:
c = icndb[COLL_JOKES].aggregate([
      {"$limit": 10},
])

pd.DataFrame(c).sample(10)

,_id,joke,categories
8,323,Chuck Norris can be unlocked on the hardest level of Tekken. But only Chuck Norris is skilled en...,[]
7,236,"Sticks and stones may break your bones, but a Chuck Norris glare will liquefy your kidneys.",[]
5,316,"In the medical community, death is referred to as &quot;Chuck Norris Disease&quot;",[]
1,121,"It takes 14 puppeteers to make Chuck Norris smile, but only 2 to make him destroy an orphanage.",[]
9,333,"Chuck Norris was banned from competitive bullriding after a 1992 exhibition in San Antonio, when...",[]
4,57,"Chuck Norris is ten feet tall, weighs two-tons, breathes fire, and could eat a hammer and take a...",[]
6,149,Chuck Norris proved that we are alone in the universe. We weren't before his first space expedit...,[]
2,235,Chuck Norris once kicked a horse in the chin. Its descendants are known today as Giraffes.,[]
0,535,Chuck Norris uses canvas in IE.,[nerdy]
3,140,"Chuck Norris built a better mousetrap, but the world was too frightened to beat a path to his door.",[]
